# Исследовательский проект по НИС «Анализ данных в Python»
<div style="text-align: right">
    Проект подготовлен студентами БПИ228
    <br>
    Лысиным Кириллом и Гакал Анжеликой
</div>

## Часть 1.

## Часть 2.

In [6]:
#import requests
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import os.path


    
driver = webdriver.Chrome()

with open('visited_links.txt', 'w+') as f:
    visited_links = set(f.readlines())


def get_item_info(url):
    # Можно сделать словарь вместо массива для более удобного обращения к элементам
    item_info = []
    info_dict = {}
    
    # Тут должны вызываться методы, возвращающие строки с нужными значениями, которые будут добавляться в item_info
    soup = get_soup(url)
    if(soup == None):
        return
    
    general_info = str(soup.findAll('meta', {'property':"og:description"})[0])

    #через список
    item_info.append(get_item_name(general_info))
    item_info.append(get_item_price(general_info))
    item_info.append(get_item_category(general_info))
    item_info.append(get_item_region(soup))
    item_info.append(get_item_placement_time(soup))
    item_info.append(get_seller_name(soup))
    item_info.append(get_number_of_ads(soup))
    item_info.append(get_seller_rating(soup))
    item_info.append(get_number_of_views(soup))
    item_info.append(get_number_of_images(soup))
    item_info.append(url)
    
    #через словарь
    info_dict['Name'] = get_item_name(general_info)
    info_dict['Price (in rubles)'] = get_item_price(general_info)
    info_dict['Category'] = get_item_category(general_info)
    info_dict['Region'] = get_item_region(soup)
    info_dict['Time/date of placement'] = get_item_placement_time(soup)
    info_dict['Seller\'s name'] = get_seller_name(soup)
    info_dict['Number of seller\'s ads'] = get_number_of_ads(soup)
    info_dict['Seller rating'] = get_seller_rating(soup)
    info_dict['Number of views'] = get_number_of_views(soup)
    info_dict['Number of images'] = get_number_of_images(soup)
    info_dict['Link'] = url
    
    return item_info, info_dict

def get_soup(url, counter = 0):

    time.sleep(1)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

def get_item_name(info):
    try:
        name = info[15:info.find('–') - 1]
        return name
    except:
        return None

def get_item_price(info):
    if not('Цена: ' in info):
        return 0
    price = int(info[info.find('Цена: ') + 6:info.find('руб.') - 1].replace(' ', ''))
    return price

def get_item_category(info):
    try:
        category = info[info.find('раздела «') + 9:info.find('».')]
        return category
    except:
        return None

def get_item_region(soup):
    try:
        region = list(soup.findAll('span', class_='sc-cOxWqc hNYaaO'))[0].text
        return region
    except:
        return None

def get_item_placement_time(soup):
    try:
        time_of_placement = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return time_of_placement[-1].text
    except:
        return None

def get_seller_name(soup):
    try:
        name_of_seller = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        name_of_seller = name_of_seller[:name_of_seller.find('(') - 1]
        return name_of_seller
    except:
        return None

def get_number_of_ads(soup):
    try:
        n = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        n = int(n[n.find('(')+1:n.find(' объявл')])
        return n
    except:
        return None

def get_seller_rating(soup):
    try:
        rating = soup.find('span', class_="sc-cOxWqc eDoIYl")
        return float(rating.text.replace(',', '.')) if rating != None else None
    except:
        return None

def get_number_of_views(soup):
    try:
        number_of_views = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return int(number_of_views[-2].text)
    except:
        return None

def get_number_of_images(soup):
    try:
        images = list(soup.findAll('img', class_="sc-fBnnfK"))
        return len(images)
    except:
        return None

def get_links(scroll_count):
    
    url = r'https://youla.ru/'
    driver.get(url)
    time.sleep(1)
    
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    links = [str(i.a) for i in soup.findAll('span', class_="sc-llGDqb sc-gqgnwQ fEAASo hZGRky")]
    links = [r'https://youla.ru' + s[s.find(r'href="') + 6 : s.find(r'" rel')] for s in links]
    return links



items = []
for i in range(10):
    links = get_links(1)
    print("Processing data. Please wait...")
    for i in range(len(links)):
        if(int(i * 100/len(links))//5 > int((i - 1) * 100/len(links))//5):
            print(f'\t{((i * 100/len(links))//5)*5}%')
        link = links[i]
        if(link in visited_links):
            continue
        items.append(get_item_info(link)[1])
        visited_links.add(link)
    print('\t100.0%')

    df = pd.DataFrame(items)
    df = df.set_index('Name')

    if(os.path.isfile('items.xlsx')):
        prev_df = pd.read_excel('items.xlsx', index_col = 0)
        df = pd.concat([prev_df, df])

    df.to_excel('items.xlsx')



with open('visited_links.txt', 'a') as f:
    for line in visited_links:
        f.write(f"{line}\n")


# Есть какая-то проблема с дубликатами, надо поправить

driver.quit()
df

Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55.0%
	60.0%
	65.0%
	70.0%
	75.0%
	80.0%
	85.0%
	90.0%
	95.0%
	100.0%
Processing data. Please wait...
	0.0%
	5.0%
	10.0%
	15.0%
	20.0%
	25.0%
	30.0%
	35.0%
	40.0%
	45.0%
	50.0%
	55

,Price (in rubles),Category,Region,Time/date of placement,Seller's name,Number of seller's ads,Seller rating,Number of views,Number of images,Link
Name,,,,,,,,,,
Шведская стенка,3000,Спорт и отдых,Москва,29.12.2022,Лидия Т.,1.0,5.0,238,4,https://youla.ru/moskva/sport-otdyh/trenazhery...
Продается СВЕРЛИЛЬНЫЙ СТАНОК EINHELL SB 1625,6000,Стройматериалы и инструменты,Москва,108,Алексей П.,1.0,NaN,16,4,https://youla.ru/moskva/remont-i-stroitelstvo/...
Бейсболки найк,1500,Мужской гардероб,Москва,Сегодня в 4:59,Серега П.,5.0,5.0,27,3,https://youla.ru/moskva/muzhskaya-odezhda/golo...
Palit Rtx2070 Jetstream 8gb,20000,Компьютерная техника,Москва,8 января,Дмитрий П.,1.0,NaN,12,5,https://youla.ru/moskva/kompyutery/komplektuyu...
"Land Rover Freelander, 2011",1089000,Легковые автомобили,Химки,Позавчера в 1:38,Менеджер,309.0,NaN,11,8,https://youla.ru/himki/auto/s-probegom/land-ro...
...,...,...,...,...,...,...,...,...,...,...
Роял Канин Urinary S/O сухой корм для кошек,500,Животные,Москва,30.12.2022,Ольга О.,31.0,5.0,115,0,https://youla.ru/moskva/zhivotnye/koshki/roial...
Женские кроссовки Nike Air Max 2021 ✔,2900,Женский гардероб,Москва,Сегодня в 6:28,Алина Л,1041.0,4.1,51,4,https://youla.ru/moskva/zhenskaya-odezhda/obuv...
Оперативная память ddr3 32gb 1866,4000,Компьютерная техника,Москва,Вчера в 20:11,Евгений А.,1.0,5.0,3,4,https://youla.ru/moskva/kompyutery/komplektuyu...


In [55]:
from selenium import webdriver
driver = webdriver.Chrome()
def get_soup(url, counter = 0):
    print(counter)
    time.sleep(1)
    #page = requests.get(url)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

driver.quit()
print(get_soup(r'https://youla.ru/moskva/auto/s-probegom/kia-rio-2015-632f29576657d33ea5424a2c'))

0
1
2
3
None
